In [1]:
%cd /kaggle/input/vietnam-textdetect-ocr/OCR-pipeline/OCR-pipeline/vietnamese-ocr

/kaggle/input/vietnam-textdetect-ocr/OCR-pipeline/OCR-pipeline/vietnamese-ocr


In [2]:
!pip install -q -r requirement.txt

In [3]:
!pip install -q Pillow==9.5.0

In [4]:
!pip install -q paddleocr
!pip install -q paddlepaddle

In [5]:
!sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2.19_amd64.deb

Selecting previously unselected package libssl1.1:amd64.
(Reading database ... 123110 files and directories currently installed.)
Preparing to unpack libssl1.1_1.1.1f-1ubuntu2.19_amd64.deb ...
Unpacking libssl1.1:amd64 (1.1.1f-1ubuntu2.19) ...
Setting up libssl1.1:amd64 (1.1.1f-1ubuntu2.19) ...
Processing triggers for libc-bin (2.35-0ubuntu3.8) ...


In [6]:
!pip install gdown

In [7]:
import paddleocr

In [8]:
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image

from vietocr.vietocr.tool.predictor import Predictor
from vietocr.vietocr.tool.config import Cfg

from PaddleOCR import PaddleOCR, draw_ocr

In [9]:
import cv2
from PIL import Image
import matplotlib.pyplot as plt

FONT = './PaddleOCR/doc/fonts/latin.ttf'

def predict(recognitor, detector, img_path, padding=4, dpi=100):
    # Load image
    img = cv2.imread(img_path)

    # Text detection
    result = detector.ocr(img_path, cls=False, det=True, rec=False)
    result = result[:][:][0]

    # Filter Boxes
    boxes = []
    for line in result:
        boxes.append([[int(line[0][0]), int(line[0][1])], [int(line[2][0]), int(line[2][1])]])
    
    # Calculate box areas and sort by area (descending)
    box_areas = [(box, (box[1][0] - box[0][0]) * (box[1][1] - box[0][1])) for box in boxes]
    sorted_boxes = sorted(box_areas, key=lambda x: x[1], reverse=True)
    
#     if len(sorted_boxes) <= 5:
#         top_boxes = [box for box, _ in sorted_boxes]  # Take all boxes if 6 or fewer
#     else:
#         top_boxes = [box for box, _ in sorted_boxes[:5]]  # Take the top 6 boxes
    top_boxes = [box for box, _ in sorted_boxes]
    # Add padding to boxes
    for box in top_boxes:
        box[0][0] = max(box[0][0] - padding, 0)
        box[0][1] = max(box[0][1] - padding, 0)
        box[1][0] = min(box[1][0] + padding, img.shape[1])
        box[1][1] = min(box[1][1] + padding, img.shape[0])

    # Text recognition
    texts = []
    for box in top_boxes:
        cropped_image = img[box[0][1]:box[1][1], box[0][0]:box[1][0]]
        try:
            cropped_image = Image.fromarray(cropped_image)
        except:
            continue
#         st_time = time.time()
        rec_result = recognitor.predict(cropped_image)
#         print((time.time() - st_time) * 1000, 'ms')

        texts.append(rec_result)

    # # Convert boxes to draw
    # def get_rectangle_points(x1, y1, x2, y2):
    #     return [(x1, y1), (x2, y1), (x2, y2), (x1, y2)]

    # _boxes = [get_rectangle_points(boxe[0][0], boxe[0][1], boxe[1][0], boxe[1][1]) for boxe in top_boxes]

    # # Draw boxes and texts
    # img = draw_ocr(img, _boxes, texts, scores=None, font_path=FONT)

    # # Save image
    # img_name = img_path.split('/')[-1]

    # # Display image
    # img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    # height, width, depth = img.shape

    # # What size does the figure need to be in inches to fit the image?
    # figsize = width / float(dpi), height / float(dpi)

    # # Create a figure of the right size with one axes that takes up the full figure
    # fig = plt.figure(figsize=figsize)
    # ax = fig.add_axes([0, 0, 1, 1])

    # # Hide spines, ticks, etc.
    # ax.axis('off')

    # # Display the image.
    # ax.imshow(img, cmap='gray')

    # plt.show()

    # return top_boxes, texts

    return texts



In [10]:
# Config of PaddleOCR
detector = PaddleOCR(use_angle_cls=False, lang="vi", use_gpu=True)

download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:10<00:00, 373kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/latin_PP-OCRv3_rec_infer.tar to /root/.paddleocr/whl/rec/latin/latin_PP-OCRv3_rec_infer/latin_PP-OCRv3_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:17<00:00, 567kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:14<00:00, 148kiB/s]

[2024/10/11 23:07:29] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/latin/latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25,

In [11]:
# Configure of VietOCR
config = Cfg.load_config_from_name('vgg_transformer')
# config = Cfg.load_config_from_file('vietocr/config.yml')
config['weights'] = '/kaggle/input/vietnam-textdetect-ocr/OCR-pipeline/OCR-pipeline/vietnamese-ocr/vietocr/transformerocr.pth'

config['cnn']['pretrained'] = True
config['predictor']['beamsearch'] = True
config['device'] = 'cuda:0' # mps
# config['device'] = 'cpu' # mps

recognitor = Predictor(config)

Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth
100%|██████████| 548M/548M [00:02<00:00, 202MB/s]


In [12]:
import os
import csv
import time
from tqdm import tqdm
import re
# Define the image directory and output CSV file path
image_directory = '/kaggle/input/aic-frames/output'
output_csv = '/kaggle/working/ocr_results.csv'
temp_output_csv = '/kaggle/working/temp_ocr_results.csv'  # Temporary file

# Collect all image paths in the directory and sort them
# images_path = sorted([os.path.join(image_directory, img) for img in os.listdir(image_directory) if img.lower().endswith(('.png', '.jpg', '.jpeg'))])

# Regular expression to match image files with names starting with "L13" or greater
prefix_pattern = re.compile(r'^L(1[3-9]|[2-9][0-9])')

# Collect image paths that match the pattern and have valid image extensions
images_path = sorted([
    os.path.join(image_directory, img) 
    for img in os.listdir(image_directory) 
    if img.lower().endswith(('.png', '.jpg', '.jpeg')) #and prefix_pattern.match(img)
])

print(len(images_path))
images_path = images_path[75000:90000]

# Open the temporary CSV file for writing
with open(temp_output_csv, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['image_path', 'texts'])  # Write the initial header row

    # Iterate over each image path and process it, with a progress bar
    for img_path in tqdm(images_path, desc="Processing images", unit="image"):
        # Call the predict function on the current image
        texts = predict(recognitor, detector, img_path, padding=2, dpi=100)
        
        # Prepare the row for the CSV file
        # Join the texts into a single string with a temporary delimiter ("|")
        row = [img_path, "|".join(texts)]
        
        # Write the row to the temporary CSV file
        writer.writerow(row)

print(f"Temporary results saved to {temp_output_csv}")


110733


Processing images: 100%|██████████| 15000/15000 [1:25:09<00:00,  2.94image/s]

Temporary results saved to /kaggle/working/temp_ocr_results.csv


In [13]:
import csv

# Open the temporary CSV file for reading
with open(temp_output_csv, mode='r', encoding='utf-8') as infile:
    reader = csv.reader(infile)
    headers = next(reader)  # Skip the header row

    # Create a list to store the rows with the new structure
    processed_rows = []
    max_texts_count = 0

    # Process each row to split the 'texts' field into separate columns
    for row in reader:
        image_path = row[0]
        texts = row[1].split("|")
        max_texts_count = max(max_texts_count, len(texts))
        processed_rows.append([image_path] + texts)

# Update headers to include columns for each text
headers = ['image_path'] + [f'text_{i}' for i in range(max_texts_count)]

# Open the final output CSV file for writing
with open(output_csv, mode='w', newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(headers)  # Write the updated header row

    # Write the processed rows, padding rows with empty strings where necessary
    for row in processed_rows:
        row += [''] * (max_texts_count - len(row) + 1)
        writer.writerow(row)

print(f"Final results saved to {output_csv}")


Final results saved to /kaggle/working/ocr_results.csv
